In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
from sqlalchemy import inspect
import sys
import sqlalchemy.ext.declarative as sqld

In [2]:
sys.path.append('..')

In [3]:
from modules.transforms import *

In [4]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [5]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [7]:
pop=pd.read_sql_table('producto1_Covid_19', schema='producto1',con=cnx) 
pop=pop.groupby('Region')['Poblacion'].sum()
popTOT=pop.sum()

In [8]:
popTOT

19458310.0

In [9]:
#FALLECIDOS DEIS
s="DEIS"
n="DEIS_2020"
DEIS=pd.read_sql_table(n, con=cnx,schema=s)
DEIS=DEIS[DEIS['COVID'].notnull()]
master=DEIS.groupby('FECHA_DEF')['index'].count().reset_index()
master.columns=['Fecha','Fallecidos DEIS Diarios']
master=master.merge(DEIS[DEIS['COVID']=='Covid PCR+'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (PCR+)'}),on='Fecha')
master=master.merge(DEIS[DEIS['COVID']=='Covid Sospechoso'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (Covid Sospechoso)'}),on='Fecha')
master['Fallecidos DEIS (PCR+) Acumulado']=master['Fallecidos DEIS Diarios (PCR+)'].cumsum()
master['Fallecidos DEIS (Covid Sospechoso) Acumulado']=master['Fallecidos DEIS Diarios (Covid Sospechoso)'].cumsum()
master['Fallecidos DEIS Acumulado']=master['Fallecidos DEIS Diarios'].cumsum()
master['Fallecidos DEIS (PCR+) x Millón/Hab.']=master['Fallecidos DEIS (PCR+) Acumulado']/(popTOT/1000000)
master['Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.']=master['Fallecidos DEIS (Covid Sospechoso) Acumulado']/(popTOT/1000000)
master['Fallecidos DEIS x Millón/Hab.']=master['Fallecidos DEIS Acumulado']/(popTOT/1000000)

In [44]:
#casos nuevos según fecha de inicio de síntomas
s="producto53"
n="producto53_confirmados_"
nuevos=pd.read_sql_table(n, con=cnx,schema=s)
nuevos=pd.DataFrame(nuevos.groupby('fecha')['Promedio'].sum())
nuevos.columns=['Casos Nuevos (ICOVID)']
nuevos['Incidencia Nuevos (ICOVID) x 100 K Hab.']=nuevos['Casos Nuevos (ICOVID)']/popTOT*100000
nuevos['Casos Acumulados (ICOVID)']=nuevos['Casos Nuevos (ICOVID)'].cumsum()

In [51]:
#casos MINSAL
s="producto5"
n="producto5_TotalesNacionales_T"
minsal=pd.read_sql_table(n, con=cnx,schema=s)

In [52]:
minsal

,index,Fecha,Casos nuevos con sintomas,Casos totales,Casos recuperados,Fallecidos,Casos activos,Casos nuevos sin sintomas,Casos nuevos totales,Casos activos por FD,Casos activos por FIS,Casos recuperados por FIS,Casos recuperados por FD,Casos confirmados recuperados,Casos activos confirmados,Casos probables acumulados,Casos activos probables,Casos nuevos sin notificar
0,0,2020-03-02,1.0,1.0,0.0,0.0,1.0,NaN,1.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,1,2020-03-03,0.0,1.0,0.0,0.0,1.0,NaN,0.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,2,2020-03-04,2.0,3.0,0.0,0.0,3.0,NaN,2.0,3.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,3,2020-03-05,1.0,4.0,0.0,0.0,4.0,NaN,1.0,4.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,4,2020-03-06,1.0,5.0,0.0,0.0,5.0,NaN,1.0,5.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,198,2020-09-16,918.0,439287.0,NaN,12058.0,15377.0,353.0,1305.0,24548.0,15377.0,411852.0,402681.0,411853.0,NaN,56269.0,3658.0,34.0
199,199,2020-09-17,1283.0,441150.0,NaN,12142.0,15080.0,540.0,1860.0,24649.0,15080.0,413928.0,404359.0,413928.0,NaN,56677.0,3657.0,37.0
200,200,2020-09-18,1097.0,442827.0,NaN,12199.0,14647.0,510.0,1673.0,24358.0,14647.0,415981.0,406270.0,415981.0,NaN,57015.0,3627.0,66.0
201,201,2020-09-19,1230.0,444674.0,NaN,12254.0,14319.0,576.0,1848.0,24240.0,14319.0,418101.0,408180.0,418101.0,NaN,57317.0,3543.0,42.0


In [45]:
#casos activos según fecha de inicio de síntomas
s="producto53"
n="producto53_confirmados_regionale_casos_activos"
activos=pd.read_sql_table(n, con=cnx,schema=s)
activos=pd.DataFrame(activos.groupby('fecha')['Promedio'].sum())
activos.columns=['Casos Activos (ICOVID)']
activos['Incidencia Activos (ICOVID) x 100 K Hab.']=activos['Casos Activos (ICOVID)']/popTOT*100000

In [46]:
icovid=pd.concat([nuevos,activos],axis=1)
icovid=icovid.reset_index()
icovid=icovid.rename(columns={'fecha':'Fecha'})

In [47]:
master.merge(icovid,on='Fecha',how='left')

,Fecha,Fallecidos DEIS Diarios,Fallecidos DEIS Diarios (PCR+),Fallecidos DEIS Diarios (Covid Sospechoso),Fallecidos DEIS (PCR+) Acumulado,Fallecidos DEIS (Covid Sospechoso) Acumulado,Fallecidos DEIS Acumulado,Fallecidos DEIS (PCR+) x Millón/Hab.,Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.,Fallecidos DEIS x Millón/Hab.,Casos Nuevos (ICOVID),Incidencia Nuevos (ICOVID) x 100 K Hab.,Casos Acumulados (ICOVID),Casos Activos (ICOVID),Incidencia Activos (ICOVID) x 100 K Hab.
0,2020-03-20,2,1,1,1,1,2,0.051392,0.051392,0.102784,302.825,1.556276,2156.545,2131.440,10.953880
1,2020-03-21,2,1,1,2,2,4,0.102784,0.102784,0.205568,287.060,1.475257,2443.605,2412.785,12.399766
2,2020-03-25,2,1,1,3,3,6,0.154176,0.154176,0.308352,310.735,1.596927,3662.005,3541.065,18.198215
3,2020-03-26,2,1,1,4,4,8,0.205568,0.205568,0.411135,316.430,1.626195,3978.435,3800.430,19.531141
4,2020-03-27,4,1,3,5,7,12,0.256960,0.359743,0.616703,316.900,1.628610,4295.335,4025.995,20.690363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2020-08-15,90,48,42,10645,4015,14660,547.067037,206.338577,753.405614,1379.415,7.089079,371488.135,24186.085,124.296946
146,2020-08-16,83,45,38,10690,4053,14743,549.379674,208.291470,757.671144,1333.670,6.853987,372821.805,23893.840,122.795042
147,2020-08-17,96,52,44,10742,4097,14839,552.052054,210.552715,762.604769,1712.360,8.800148,374534.165,23784.515,122.233200
148,2020-08-18,74,39,35,10781,4132,14913,554.056339,212.351432,766.407771,1697.600,8.724293,376231.765,23707.990,121.839923
